In [1]:
import numpy as np
import os
import sys
import argparse
import glob
import time
sys.path.append("/home/jungmo/apps/caffe/python") # caffe path

import caffe
from shutil import copyfile

#GPUID = 1
BIRD_ROOT = "/home/jungmo/Documents/bird/caffe/"

NPY_PATH = os.path.join(BIRD_ROOT,"model","new_256_200", "db","mean.npy")
CAFFE_BIN = "/home/jungmo/apps/caffe/build/tools/"
network = 'googLeNet'
IMG_SIZE = 224

def binaryproto_to_npy():
    blob = caffe.proto.caffe_pb2.BlobProto()
    data = open( BINARYPROTO_PATH , 'rb' ).read()
    blob.ParseFromString(data)
    arr = np.array( caffe.io.blobproto_to_array(blob) )
    out = arr[0]
    np.save( NPY_PATH, out )

def evaluate_model(dst):
        input_file = os.path.join(TEST)
        output_file = os.path.join(BIRD_ROOT, "db/output.npy")
        model_def = os.path.join(BIRD_ROOT, "model_prototxt/"+network+"/deploy.prototxt")
        pretrained_model = os.path.join(BIRD_ROOT, "model/new_256_200/googlenet_iter_60000.caffemodel")
        gpu = True
        images_dim='224,224'
        mean_file = NPY_PATH
        ext='bmp'
        channel_swap='2,1,0'
        raw_scale = 255.0
        center_only=True
        input_scale = None
        image_dims = [int(s) for s in images_dim.split(',')]

        mean, channel_swap = None, None
        if mean_file:
            mean = np.load(mean_file)
        if channel_swap:
            channel_swap=[int(s) for s in args.channel_swap.split(',')]
        caffe.set_mode_gpu()
        #caffe.set_device(GPUID)
        # Make classifier.
        classifier = caffe.Classifier(model_def, pretrained_model,
                image_dims=image_dims, mean=mean,
                input_scale=input_scale, raw_scale=raw_scale,
                channel_swap=channel_swap)

        # Load numpy array (.npy), directory glob (*.jpg), or image file.
        imgs_list = tuple(open(input_file,'r'))
        print("Loading file: %s" % input_file)
        inputs =[caffe.io.load_image(im_f)
                     for im_f in glob.glob(input_file + '/*.' + ext)]

        print("Classifying %d inputs." % len(imgs_list))

        BATCH_SIZE = 100

        # Classify.
        confusion_matrix = np.zeros((4,4),dtype=np.int32)
        accuracy = 0.0
        start = time.time()
        for i in range(0, len(imgs_list), BATCH_SIZE):
            if i > 0 and i % (BATCH_SIZE * 25) == 0:
                print("Finished %d imgs." % (i))
                cur_accuracy = accuracy / i * 100
                print("Accuracy: %.2f " % cur_accuracy)
                print("Confusion Matrix:")
                print confusion_matrix
            imgs = [imgs_list[i + j].split(' ')[0] for j in range(min(BATCH_SIZE,len(imgs_list)-i))]
            gts = [int(imgs_list[i + j].split(' ')[1]) for j in range(min(BATCH_SIZE,len(imgs_list)-i))]
            inputs = [caffe.io.load_image(path) for path in imgs]
            predictions = classifier.predict(inputs, not center_only)
            for j in range(len(imgs)):
                probs = predictions[j]
                pred = np.argmax(probs)
                confusion_matrix[gts[j]][pred] += 1
                if pred == gts[j]:
                    accuracy += 1.0

        accuracy = accuracy / len(imgs_list) * 100

        print("Done in %.2f s." % (time.time() - start))
        # Save
        print("Accuracy: %.2f " % accuracy)
        print("Confusion Matrix:")
        print confusion_matrix
        np.save(dst, confusion_matrix)

In [2]:

'''    
print "Make DB : Training set"
os.system(os.path.join(CAFFE_BIN,"convert_imageset.bin")+" -backend=\"lmdb\" -shuffle=true -resize_height "+str(IMG_SIZE)+" -resize_width "+str(IMG_SIZE)+ " / " + TRAIN + " " + BIRD_ROOT + "db/train_imageData_lmDB")

print "Make DB : Test set"
os.system(os.path.join(CAFFE_BIN,"convert_imageset.bin")+" -backend=\"lmdb\" -shuffle=true -resize_height "+str(IMG_SIZE)+" -resize_width "+str(IMG_SIZE) + " / " + TEST + " " + BIRD_ROOT + "db/val_imageData_lmDB")

print "Make image mean file"
os.system(os.path.join(CAFFE_BIN,"compute_image_mean.bin")+
               " -backend=lmdb "+ BIRD_ROOT + "db/train_imageData_lmDB " + BIRD_ROOT+ "db/mean.binaryproto")
print "Image mean file .binaryproto to .npy"
binaryproto_to_npy()   

print "Training model"
os.system(os.path.join(CAFFE_BIN,"caffe.bin")+" train "
               "-solver="+BIRD_ROOT+"model/"+network+"/solver.prototxt "+" -weights="+BIRD_ROOT+"model/"+network+"/bvlc_googlenet.caffemodel "+ "-gpu all")
'''
print "Evaluate model"
TEST = "/home/jungmo/Documents/bird/caffe/bird_no_adole/test//test_256_200.txt"
evaluate_model("test_256_200.npy")


Evaluate model
Loading file: /home/jungmo/Documents/bird/caffe/bird_no_adole/test//test_256_200.txt
Classifying 21586 inputs.
Finished 2500 imgs.
Accuracy: 97.76 
Confusion Matrix:
[[2444   32   16    8]
 [   0    0    0    0]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 5000 imgs.
Accuracy: 97.66 
Confusion Matrix:
[[4883   50   39   28]
 [   0    0    0    0]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 7500 imgs.
Accuracy: 97.68 
Confusion Matrix:
[[6594   64   48   32]
 [   5  732    7   18]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 10000 imgs.
Accuracy: 95.91 
Confusion Matrix:
[[8602   71   78  218]
 [   8  989    8   26]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 12500 imgs.
Accuracy: 95.36 
Confusion Matrix:
[[9491   76   89  311]
 [   8  989    8   26]
 [   1    9 1440   52]
 [   0    0    0    0]]
Finished 15000 imgs.
Accuracy: 95.28 
Confusion Matrix:
[[9491   76   89  311]
 [   8  989    8   26]
 [   7   22 3812  161]
 [   0 

In [3]:
TEST = "/home/jungmo/Documents/bird/caffe/bird_no_adole/test//test_256_150.txt"
evaluate_model("test_256_150.npy")

Loading file: /home/jungmo/Documents/bird/caffe/bird_no_adole/test//test_256_150.txt
Classifying 21586 inputs.
Finished 2500 imgs.
Accuracy: 98.40 
Confusion Matrix:
[[2460   15   19    6]
 [   0    0    0    0]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 5000 imgs.
Accuracy: 98.42 
Confusion Matrix:
[[4921   33   32   14]
 [   0    0    0    0]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 7500 imgs.
Accuracy: 98.08 
Confusion Matrix:
[[6633   46   40   19]
 [  26  723    8    5]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 10000 imgs.
Accuracy: 97.11 
Confusion Matrix:
[[8734   48   61  126]
 [  37  977   11    6]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 12500 imgs.
Accuracy: 95.57 
Confusion Matrix:
[[9678   50   67  172]
 [  37  977   11    6]
 [  49    7 1291  155]
 [   0    0    0    0]]
Finished 15000 imgs.
Accuracy: 93.99 
Confusion Matrix:
[[9678   50   67  172]
 [  37  977   11    6]
 [ 131   16 3444  411]
 [   0    0    0    0]

In [4]:
TEST = "/home/jungmo/Documents/bird/caffe/bird_no_adole/test//test_256_140.txt"
evaluate_model("test_256_140.npy")

Loading file: /home/jungmo/Documents/bird/caffe/bird_no_adole/test//test_256_140.txt
Classifying 21586 inputs.
Finished 2500 imgs.
Accuracy: 98.52 
Confusion Matrix:
[[2463   19   10    8]
 [   0    0    0    0]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 5000 imgs.
Accuracy: 98.48 
Confusion Matrix:
[[4924   37   24   15]
 [   0    0    0    0]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 7500 imgs.
Accuracy: 98.12 
Confusion Matrix:
[[6637   44   40   17]
 [  30  722    6    4]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 10000 imgs.
Accuracy: 97.08 
Confusion Matrix:
[[8731   48   62  128]
 [  39  977   10    5]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 12500 imgs.
Accuracy: 96.14 
Confusion Matrix:
[[9686   49   69  163]
 [  39  977   10    5]
 [  24    7 1355  116]
 [   0    0    0    0]]
Finished 15000 imgs.
Accuracy: 95.09 
Confusion Matrix:
[[9686   49   69  163]
 [  39  977   10    5]
 [  79   12 3601  310]
 [   0    0    0    0]

In [9]:
TEST = "/home/jungmo/Documents/bird/caffe/bird_no_adole/test//test_256_130.txt"
evaluate_model("test_256_130.npy")

Loading file: /home/jungmo/Documents/bird/caffe/bird_no_adole/test//test_256_130.txt
Classifying 21586 inputs.
Finished 2500 imgs.
Accuracy: 98.64 
Confusion Matrix:
[[2466   16   12    6]
 [   0    0    0    0]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 5000 imgs.
Accuracy: 98.66 
Confusion Matrix:
[[4933   25   29   13]
 [   0    0    0    0]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 7500 imgs.
Accuracy: 98.24 
Confusion Matrix:
[[6644   37   38   19]
 [  28  724    6    4]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 10000 imgs.
Accuracy: 97.06 
Confusion Matrix:
[[8733   41   62  133]
 [  39  973   11    8]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 12500 imgs.
Accuracy: 96.39 
Confusion Matrix:
[[9674   43   72  178]
 [  39  973   11    8]
 [  14    5 1402   81]
 [   0    0    0    0]]
Finished 15000 imgs.
Accuracy: 95.88 
Confusion Matrix:
[[9674   43   72  178]
 [  39  973   11    8]
 [  37   11 3735  219]
 [   0    0    0    0]

In [10]:
TEST = "/home/jungmo/Documents/bird/caffe/bird_no_adole/test//test_256_120.txt"
evaluate_model("test_256_120.npy")

Loading file: /home/jungmo/Documents/bird/caffe/bird_no_adole/test//test_256_120.txt
Classifying 21586 inputs.
Finished 2500 imgs.
Accuracy: 98.64 
Confusion Matrix:
[[2466   11   14    9]
 [   0    0    0    0]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 5000 imgs.
Accuracy: 98.58 
Confusion Matrix:
[[4929   21   36   14]
 [   0    0    0    0]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 7500 imgs.
Accuracy: 98.25 
Confusion Matrix:
[[6648   28   45   17]
 [  13  721    9   19]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 10000 imgs.
Accuracy: 96.13 
Confusion Matrix:
[[8642   31   76  220]
 [  20  971   12   28]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 12500 imgs.
Accuracy: 94.23 
Confusion Matrix:
[[9516   33   93  325]
 [  20  971   12   28]
 [  39    3 1292  168]
 [   0    0    0    0]]
Finished 15000 imgs.
Accuracy: 92.95 
Confusion Matrix:
[[9516   33   93  325]
 [  20  971   12   28]
 [  95    8 3455  444]
 [   0    0    0    0]

In [11]:
TEST = "/home/jungmo/Documents/bird/caffe/bird_no_adole/test//test_256_110.txt"
evaluate_model("test_256_110.npy")

Loading file: /home/jungmo/Documents/bird/caffe/bird_no_adole/test//test_256_110.txt
Classifying 21586 inputs.
Finished 2500 imgs.
Accuracy: 98.80 
Confusion Matrix:
[[2470    8   17    5]
 [   0    0    0    0]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 5000 imgs.
Accuracy: 98.80 
Confusion Matrix:
[[4940   19   33    8]
 [   0    0    0    0]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 7500 imgs.
Accuracy: 98.36 
Confusion Matrix:
[[6664   25   40    9]
 [  33  713    9    7]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 10000 imgs.
Accuracy: 96.34 
Confusion Matrix:
[[8673   28   55  213]
 [  51  961   10    9]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 12500 imgs.
Accuracy: 95.21 
Confusion Matrix:
[[9556   30   67  314]
 [  51  961   10    9]
 [  16    0 1384  102]
 [   0    0    0    0]]
Finished 15000 imgs.
Accuracy: 94.66 
Confusion Matrix:
[[9556   30   67  314]
 [  51  961   10    9]
 [  57    3 3682  260]
 [   0    0    0    0]

In [12]:
TEST = "/home/jungmo/Documents/bird/caffe/bird_no_adole/test//test_256_100.txt"
evaluate_model("test_256_100.npy")

Loading file: /home/jungmo/Documents/bird/caffe/bird_no_adole/test//test_256_100.txt
Classifying 21586 inputs.
Finished 2500 imgs.
Accuracy: 99.20 
Confusion Matrix:
[[2480   12    7    1]
 [   0    0    0    0]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 5000 imgs.
Accuracy: 99.04 
Confusion Matrix:
[[4952   22   23    3]
 [   0    0    0    0]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 7500 imgs.
Accuracy: 98.49 
Confusion Matrix:
[[6683   25   26    4]
 [  47  704    3    8]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 10000 imgs.
Accuracy: 98.07 
Confusion Matrix:
[[8858   27   33   51]
 [  69  949    4    9]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 12500 imgs.
Accuracy: 94.34 
Confusion Matrix:
[[9822   29   39   77]
 [  69  949    4    9]
 [ 278    0 1021  203]
 [   0    0    0    0]]
Finished 15000 imgs.
Accuracy: 90.55 
Confusion Matrix:
[[9822   29   39   77]
 [  69  949    4    9]
 [ 725    0 2811  466]
 [   0    0    0    0]